In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys; sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

from helpers import parse_runs, load_scores_from_run_df, plot_multi

OUTPUT_LATEX = True

plt.style.use("default")
if OUTPUT_LATEX:
    matplotlib.use("pdf")
    matplotlib.rcParams.update({
        "pgf.texsystem": "pdflatex",
        'font.family': 'serif',
        'text.usetex': True,
        'pgf.rcfonts': False,
    })


In [ ]:
# Specify the directories of the data that should be loaded.
# All subfolders are automatically analyzed
data_dirs = [
]

In [ ]:
# Load the data
runs = parse_runs(data_dirs, 
    must_include_hyperparams=["Seed"],  # Always create a seed level
    ignore_hyperparams=["save_checkpoints_every_n_epochs"]
) 
df = load_scores_from_run_df(runs, must_be_same_length=True, metric="auroc")
experiment_name = "_".join(filter(lambda x: x not in ["Epoch", "Seed"], df.index.names))
print(runs.index)
df

In [ ]:
#df.index.rename("Previous Classes", level=1, inplace=True)

In [ ]:
# Get mean score auroc for hyperparameter combination over all targets and seeds. Maximum over all epochs
scores_max_mean = df.unstack(level="Epoch").max(axis=1, level=0).unstack(level="Seed").mean(axis=1, level=0)
scores_last_mean = df.groupby(list(filter(lambda x: x != "Epoch", df.index.names))).last().unstack(level="Seed").mean(axis=1, level=0)

scores = scores_max_mean.mean(axis=1)
scores

In [ ]:
scores_max_mean

In [ ]:
if OUTPUT_LATEX:
    scores.name="AUROC Score"
    scores.to_latex(f"{experiment_name}_auroc_table.tex", float_format="%.4f")

In [ ]:
plt.figure(figsize=(10,2))
g = sns.heatmap(scores_max_mean, annot=True, fmt=".3g")
g.set_yticklabels(labels=g.get_yticklabels(), va='center')
for l in g.get_yticklabels():
    l.set_ha("right")
    l.set_rotation(0)
if OUTPUT_LATEX:
    plt.savefig(f"{experiment_name}_auroc_heatmap.pdf", bbox_inches="tight")
plt.show()

In [ ]:
group_by = ["Learning Rate", "Model Type", "Few-Shot Method"]

In [ ]:
grouped_scores = scores_max_mean.groupby(group_by).mean().mean(axis=1)
if OUTPUT_LATEX:
    grouped_scores.name="AUROC Score"
    grouped_scores.to_latex(f"{experiment_name}_auroc_table.tex", float_format="%.4f")
grouped_scores

In [ ]:
for group_name, d in scores_max_mean.groupby(group_by):
    for l in group_by:
        d = d.droplevel(l)
    #d = d.reindex(["Plane", "Car", "Bird", "Cat", "Deer", "Dog", "Frog", "Horse", "Ship", "Truck"])
    plt.figure()
    ax = plt.gca()
    g = sns.heatmap(d, annot=True, fmt=".3g", ax=ax, vmin=scores_max_mean.min().min(), vmax=scores_max_mean.max().max())
    g.set_yticklabels(labels=g.get_yticklabels(), va='center')
    for l in g.get_yticklabels():
        l.set_ha("right")
        l.set_rotation(0)
    title = " - ".join(map(str, group_name))
    ax.set_title(title)
    if OUTPUT_LATEX:
        plt.savefig(f"{title}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
# Take mean over seeds
scores_mean_seeds = df.unstack(level="Seed").mean(axis=1, level=0)#.unstack(level="Previous Class").mean(axis=1, level=0)

plot_multi(scores_mean_seeds, "AUROC Score", bottom=0.4, top=0.74, legend_loc="lower right", save=True)